In [1]:
# Import libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import csv
import os
from dotenv import load_dotenv

# Credentials
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET= os.getenv("CLIENT_SECRET")

# Set up Spotipy client
client_credentials_manager = SpotifyClientCredentials(
    client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Read input body_parts.csv file
entrada = open('body_parts.csv')
reader = csv.reader(entrada)

is_first_row = True
data = []

# Iterate through CSV rows
for row in reader:
    if is_first_row:
        is_first_row = False
        continue
    body_part = row[0]
    
    # Search for tracks related to body part on Spotify
    spotify_data = sp.search(q='track:' + body_part, type='track', market='US', limit=50)
    
    top_three_popularities = []
    top_three_items = []

# Extract information for top three tracks based on popularity
    for item in spotify_data['tracks']['items']:
        song = item['name']
        track_url = item['external_urls']['spotify']
        popularity = item['popularity']
        for name_artist in item['album']['artists']:
            artist = name_artist['name']

        if len(top_three_popularities) < 10 or popularity > min(top_three_popularities):
            if len(top_three_popularities) == 10:
                min_popularity_index = top_three_popularities.index(min(top_three_popularities))
                top_three_popularities.pop(min_popularity_index)
                top_three_items.pop(min_popularity_index)
            top_three_popularities.append(popularity)
            top_three_items.append({'body_part': body_part, 'song': song, 'artist': artist, 'track_url': track_url, 'popularity': popularity})

    data.extend(top_three_items)

# Create DataFrame
df = pd.DataFrame(data)
df.to_csv('body_tracks.csv', index=False)
